# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

## Packages

In [6]:
import json

## Settings / Constants

In [7]:
RESULTS_PATH_BASE = "../07 train classifier/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
LLM_PAPER_TITLE = {"Llama70B": "Llama2-70B", "GPT-3": "GPT-3.5-turbo"}
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [8]:
FEW_SHOT_STRATEGY = {25: "25 fixed examples", 500: "25 random examples"}

## Code

In [9]:
def format_thousands(number):
    formatted_number = "{:,}".format(number)
    return formatted_number

def round_to_three_decimals(number):
    rounded_number = round(number, 3)
    return rounded_number

In [10]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n")
    for n_real in [500, 1000, 2000]:
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)
        print(
            f"- & - & {format_thousands(n_real)} & 0 & {round_to_three_decimals(results['eval_f1_micro'])} & {round_to_three_decimals(results['eval_f1_macro'])} & {round_to_three_decimals(results['eval_accuracy'])} \\\\")
    print("\\hline")
    for few_shot_condition in SYNTH_COMBINATIONS.keys():
        for llm in LLMS:
            for freq in SYNTH_COMBINATIONS[few_shot_condition]:
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)
                # print(f"results: {absa_task}, {llm}, {few_shot_condition}, n_real: {n_real}, n_synth: {n_synth}", results)
                if absa_task == "TASD":
                    f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])} & {round_to_three_decimals(results['eval_f1_macro'])}"
                else:
                    f1_metrics = f"{round_to_three_decimals(results['eval_f1_micro'])}"
                print(
                    f"{LLM_PAPER_TITLE[llm]} & {FEW_SHOT_STRATEGY[n_real]} & {format_thousands(n_real)} & {format_thousands(n_synth)} & {f1_metrics} & {round_to_three_decimals(results['eval_accuracy'])} & {round_to_three_decimals(results['eval_accuracy'])} \\\\")
        print("\hline")

aspect_category :

- & - & 500 & 0 & 0.061 & 0.033 & 0.026 \\
- & - & 1,000 & 0 & 0.061 & 0.033 & 0.026 \\
- & - & 2,000 & 0 & 0.061 & 0.033 & 0.026 \\
\hline
Llama2-70B & 25 random examples & 500 & 500 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 random examples & 500 & 1,000 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 random examples & 500 & 1,500 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 500 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 1,000 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 random examples & 500 & 1,500 & 0.061 & 0.026 & 0.026 \\
\hline
Llama2-70B & 25 fixed examples & 25 & 475 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 fixed examples & 25 & 975 & 0.061 & 0.026 & 0.026 \\
Llama2-70B & 25 fixed examples & 25 & 1,975 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25 & 475 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25 & 975 & 0.061 & 0.026 & 0.026 \\
GPT-3.5-turbo & 25 fixed examples & 25

In [11]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"